# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: The PEFT technique to be used is LoRA (Parameter Efficient Fine Tuning)
* Model: The model used in this project is the GPT-2 model
* Evaluation approach: The evaluation approach used in the project is accuracy score
* Fine-tuning dataset: The fine-tuning dataset used in this project is the IMDB dataset

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [2]:
# pip install the datasets library
! pip install -q "datasets==2.15.0"

In [1]:
from datasets import load_dataset
# load the train and test splits of the IMDB dataset
splits = ['train', 'test']
dataset = {split: ds for split, ds in zip(splits, load_dataset('imdb', split=splits))}

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# dataset

In [3]:
for split in splits:
    dataset[split] = dataset[split].shuffle(seed=10).select(range(500))

In [4]:
dataset

{'train': Dataset({
     features: ['text', 'label'],
     num_rows: 500
 }),
 'test': Dataset({
     features: ['text', 'label'],
     num_rows: 500
 })}

In [5]:
from transformers import AutoTokenizer

# preprocess the dataset
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [6]:
def preprocess_function(examples):
    """Preprocess the imdb dataset by returning the tokenized text"""
    return tokenizer(examples["text"], truncation=True)

tokenized_ds = {}
for split in splits:
    tokenized_ds[split] = dataset[split].map(preprocess_function, batched=True)

tokenized_ds

Map: 100%|██████████| 500/500 [00:00<00:00, 1021.61 examples/s]


{'train': Dataset({
     features: ['text', 'label', 'input_ids', 'attention_mask'],
     num_rows: 500
 }),
 'test': Dataset({
     features: ['text', 'label', 'input_ids', 'attention_mask'],
     num_rows: 500
 })}

In [7]:
for i in range(1):
    print(tokenized_ds["train"][0]["text"])
    print(f"\nSentiment -----> {tokenized_ds['train'][0]['label']}")

I always feel strange and guilty saying it (because I'm a fairly well-educated non-teenager), but I actually sort of like the Olsen twins, and I respect the movies they make, even though I've never really been their target audience. "When in Rome" was a traditional Mary-Kate and Ashley movie, complete with the foreign travel, accents, motorbikes, adult romance as a "B" storyline, fashion orientation, and even the gag reel over the credits. I enjoyed myself. "When in Rome" and the other Olsen twin movies never pretend to be anything they're not; most of the time, they only premiere on video, and they never claim to be the next "Citizen Kane" or even "An Affair to Remember." My point is, people who watch this movie and expect it to be anything other than another Olsen twin movie will be disappointed.<br /><br />That said, those who ARE fans of the Olsen twins will really enjoy themselves. For those of us who've watched them since the first episodes of "Full House," it's really great to s

In [8]:
# import transformers
from transformers import AutoModelForSequenceClassification, AutoConfig

# Load the pretrained HuggingFace model
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2,
    id2label={0: "NEGATIVE", 1: "POSITIVE"},
    label2id={"NEGATIVE": 0, "POSITIVE": 1},
    )

for param in model.base_model.parameters():
    param.requires_grad = False

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.weight', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
import numpy as np
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments

def compute_metrics(eval_pred):
    """Compute the accuracy metric"""
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

training_args = TrainingArguments(
    output_dir='./results',           # output directory
    num_train_epochs=3,              # total number of training epochs
    learning_rate=2e-3,              # learning rate
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    evaluation_strategy='epoch',     # evaluate each `logging_steps`
    save_strategy='epoch',
    # warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    # logging_dir='./logs',            # directory for storing logs
    # logging_steps=10,
    load_best_model_at_end=True,
)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

trainer = Trainer(
    model=model,                         # the instantiated Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=tokenized_ds["train"],  # training dataset
    eval_dataset=tokenized_ds["test"],    # evaluation dataset
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [10]:
trainer.train()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.520207,0.752000
2,No log,0.448576,0.786000
3,No log,0.421934,0.806000


TrainOutput(global_step=96, training_loss=0.5213555892308553, metrics={'train_runtime': 57.9909, 'train_samples_per_second': 25.866, 'train_steps_per_second': 1.655, 'total_flos': 195322144416720.0, 'train_loss': 0.5213555892308553, 'epoch': 3.0})

In [11]:
trainer.evaluate()

{'eval_loss': 0.4219342768192291,
 'eval_accuracy': 0.806,
 'eval_runtime': 8.6858,
 'eval_samples_per_second': 57.566,
 'eval_steps_per_second': 3.684,
 'epoch': 3.0}

## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [12]:
from peft import get_peft_model, LoraConfig

In [13]:
model.parameters

<bound method Module.parameters of DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropou

In [14]:
config = LoraConfig(
    lora_alpha=32,
    lora_dropout=0.01,
    target_modules=["pre_classifier"],
)

In [15]:
lora_model = get_peft_model(model, config)

In [16]:
lora_model.print_trainable_parameters()

trainable params: 12,288 || all params: 66,967,298 || trainable%: 0.01834925458691793


In [17]:
def compute_metrics(eval_pred):
    """Compute the accuracy metric"""
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

training_args = TrainingArguments(
    output_dir='./lora_results',           # output directory
    num_train_epochs=3,              # total number of training epochs
    learning_rate=2e-3,              # learning rate
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    evaluation_strategy='epoch',     # evaluate each `logging_steps`
    save_strategy='epoch',
    # warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    # logging_dir='./logs',            # directory for storing logs
    # logging_steps=10,
    load_best_model_at_end=True,
)

lora_trainer = Trainer(
    model=lora_model,                         # the instantiated Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=tokenized_ds["train"],  # training dataset
    eval_dataset=tokenized_ds["test"],    # evaluation dataset
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [18]:
lora_trainer.train()

ValueError: You should supply an encoding or a list of encodings to this method that includes input_ids, but you provided ['label']

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.